In [23]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
#importa as fórmulas do arquivo aluguel_offshore.py - note que este arquivo deverá estar na mesma pasta deste notebook
from aluguel_offshore import *

In [25]:
#pasta em que estão os arquivos no seu computador.
pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore'
#pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore\janfev'

In [26]:
#Class está definida no arquivo aluguel_offhsore.py que tem que estar na mesma pasta desse notebook
ubs, morgan, pras, bdwm, intc, ticker_diffs = ImportAluguelOffshore(pasta).import_files()

130 aqrivos do ubs importados
230 aqrivos da morgan importados
113 aqrivos do Calypso PROP+ASSET importados
111 aqrivos do Calypso BD+WM importados
123 aqrivos do Calypso INTC importados


# Ajustes base

In [27]:
#Ajustes da base estão definidos no aluguel_offshore.py que auxilia nesse notebook
#Eventual quadrado rosa abaixo não significa que houve erro!
pb = get_pb(morgan, ubs)
calypso = ajuste_calypso(pras, bdwm, intc)
calypso_1 = calypso.copy()
calypso_1['acc'] = calypso_1['conta_pb'].map(dict_contas)


pb, calypso = ajuste_dias(pb, calypso)


calypso = limpa_calypso(calypso)

C:\Users\correigu\Python\aluguel_offshore\aluguel_offshore.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  morgan['data_ref'] = np.where(morgan['data_ref'].dt.weekday==6, morgan['data_ref']-datetime.timedelta(2), morgan['data_ref'])


In [28]:
#Fórmula que também está definida no arquivo aluguel_offshore.py
mapa_dict = mapeamento_tickers(pb, calypso)

C:\Users\correigu\Python\aluguel_offshore\aluguel_offshore.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  on_cusip_2['ticker_novos'])


In [29]:
ticker_diffs = ticker_diffs.rename(columns={
    'Stock':'ticker',
    'Isin':'isin',
    'Cusip':'cusip'
})

mapa_dict_tickers = mapeamento_tickers(ticker_diffs, calypso)
ticker_diffs['ticker'] = ticker_diffs['ticker'].map(mapa_dict_tickers)

C:\Users\correigu\Python\aluguel_offshore\aluguel_offshore.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  on_cusip_2['ticker_novos'])


In [30]:
#Aqui ficam os ajustes que eu já sei que são necessáiros para esse range de tempo. qualquer coisa é só adicionar nesse dicionátio
dict_ajustes = {
    'VALE':'VALE US',
     'CX':'CX US',
    'DHT':'DHT US',
    'SALT':'SALT US',
    'SIG':'SIG US',
    'ENIC':'ENIC US',
    'CPB':'CPB US',
    'SFIX':'SFIX US',
    'BSMX':'BSMX US',
    'ECH':'ECH US',
    'DRI':'DRI US',
    'SONC':'SONC US',
    'JCP':'JCP US',
    'BBY':'BBY US',
    'ALFAA':'ALFAA MM',
    'DPZ':'DPZ US',
    'XLE':'XLE US',
    'DG':'DG US',
    'BBBY':'BBBY US',
    'HOG':'HOG US',
    'TLV1':'TLEVICPO MM',
    'KIMBERA':'KIMBERA MM',
    'MFRISCOA':'MFRISCOA MM',
    'LIVEPOLC':'LIVEPOLC MM',
    'RGR':'RGR US',
    'GFINBURO':'GFINBURO MM',
    'FUNO11':'FUNO11 MM',
    'ICA*':'ICA* MM',
    'PINFRA*':'PINFRA* MM',
    'DNKN':'DNKN US',
    'VESTA*':'VESTA* MM',
    'GENTERA*':'GENTERA* MM',
    'KHC':'KHC US',
    'WU':'WU US',
    'MELI':'MELI US',
    'TGT':'TGT US',
    'CAKE':'CAKE US',
    'NFLX':'NFLX US',
    'COHU':'COHU US',
    'ORCL':'ORCL US',
    'FB':'FB US',
    'LE':'LE US',
    'CXO':'CXO US',
    'GOLD':'GOLD US',
    'FMX':'FMX US',
    'VIV':'VIV US',
    'TSU':'TSU US',
    'EC':'EC US',
    'BJRI':'BJRI US',
    'DKS':'DKS US',
    'GILD':'GILD US',
    'GFI':'GFI US',
    'MCK':'MCK US',
    'FCAU':'FCAU US',
    'ARCO':'ARCO US'
}

In [31]:
#Essa linha adicionar o dicionário acima ao dicionário do mapeamento que está 3 celulas acima (aproximadamente)
#Despois de dar essa atualizada, é necessário rodar a linha de mapeamento novamente, a que está umas 2 celulas acima
#a que tem um monte de comentários
mapa_dict.update(dict_ajustes)

In [32]:
#Aqui é aplicado o dicionário para criar uma coluna que mapeia tickers do pb pros tickers do calypso
#Pode ser que arquivos fiquem fora do mapeamento acima por causa de problemas com ticker, isin ou cusip
#Quando isso ocorrer, é necessário fazer o mapeamento manual dos faltantes, adicionando no dicionário abaixo esses tickers
#Adicionamos da seguinte forma: adiciona-se um linha no dicionário abaixo com o Enter. Então coloca-se o nome que está
#no pb entre aspas, colocamos dois pontos e colocamos o nome que está no calypso.
pb['ticker_ajuste'] = pb['ticker'].apply(lambda x: mapeamento(mapa_dict, x))

In [33]:
#Essa linha serve somente para você checkar no pb quais tickers não foram mapeados. se você colcoar um # depois de True],
#você consegue ver as linhas que não foram mapeadas e todas as informações que estão contidas nelas. Caso haja isin e cusip,
#É provavel que não tenha o ticker, por isso houve o problema. nesse caso, é preciso fazer um ajuste análogo ao que foi feito
#pro ticker LIT mais acima. se não souber se virar, contatar guilherme.correia@btgpactual.com
#Se você tirar o # entre True]#['ticker'] você poderá ver todos os tickers que apresentaram problemas e procurar por
#eles na linha abaixo
pb[pb['ticker_ajuste'].isnull()==True]['ticker'].drop_duplicates()

9                 UGP
1240            CNNEF
2306     BSMXB_OLD.MX
3626           EFGINT
11201             NaN
Name: ticker, dtype: object

In [34]:
#Aqui você procura pelos tickers que são mostrados na linha acima. você procura por uma parte do ticker somente, para ver se
#encontra algo na base do calypso. O código .contains('something') procura por esse padrão nos tickers do calypso. é como um *
# * nas linhas de procurar de qualquer programa.
#Caso você encontra de fato algo, vocêdxo adiciona à lista do dicionário dict_ajustes o ticker da linha acima e dessa linha
#isso completa o mapeamento do ticker. Não esquece de rodar novamente a linha de mapeamento depois de adicionar algo ao
#dicionário
calypso[calypso['ticker'].str.contains('ARCO')==True].ticker.drop_duplicates()

Series([], Name: ticker, dtype: object)

In [35]:
#Essa linha deve ser executada depois de terem sido feitos todos os ajustes, porque ela troca os nomes das colunas
#serve para que em ambas as bases a coluna 'ticker' sejam comunicáveis
pb = pb.rename(columns={
    'ticker':'ticker_antigo',
    'ticker_ajuste':'ticker'
})

# Batimento mensal

In [43]:
#Escolha o mês e ano do batimento
mes_ref = 'mai'
ano_ref = 2018
data = '2018-06-07'
tipo = 'month'

#### Bate os arquivos do calypso contra o pb para ver se todos os que estão no calypso estão no pb

In [44]:
calypso_month, pb_month, calypso_month_acumulado, pb_month_acumulado = \
                        get_month_or_day_batimento(calypso, pb,mes_ref, ano_ref, data, referencia = tipo)

Você está fazendo o batimento referente a mai/2018


In [45]:
calypso_month

,conta_pb,conta,agent,data_ref,quantidade,ticker,currency,acc
0,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-5331.0,RDUS US,USD,ms_cayman
1,038CDA715,ASSET,MORGAN STANLEY & CO,2018-05-01,0.0,BSMX US,USD,ms_cayman
2,75202215,ASSET,UBS SECURITIES LLC,2018-05-01,0.0,SPY US,USD,ubs_cayman
3,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-9434.0,SBLK US,USD,ms_cayman
4,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,0.0,CIB US,USD,ms_cayman
5,038CDCN39,ASSET,MORGAN STANLEY & CO,2018-05-01,-203208.0,BSMX US,USD,ms_cayman
6,75202215,ASSET,UBS SECURITIES LLC,2018-05-01,0.0,EWW US,USD,ubs_cayman
7,75202215,ASSET,UBS SECURITIES LLC,2018-05-01,0.0,EWW US,USD,ubs_cayman
8,038CDCN39,ASSET,MORGAN STANLEY & CO,2018-05-01,-2888800.0,AMX US,USD,ms_cayman
9,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,0.0,SPY US,USD,ms_cayman


In [46]:
pb_month

,conta,pb,data_ref,qt,currency,accrual_adjusted,acc,ticker
0,038CDB895,ms,2018-05-01,-567700.0,AUD,-89.03,int_c,FMG AU
1,038CDB895,ms,2018-05-01,-10200.0,MXN,-59.32,int_c,GAPB MM
2,038CDB895,ms,2018-05-01,-7620.0,USD,-3.57,int_c,TSU US
3,038CDB895,ms,2018-05-01,-400.0,USD,-0.25,int_c,BRFS US
4,038CDB895,ms,2018-05-01,-147595.0,USD,-130.25,int_c,EBR US
5,038CDB895,ms,2018-05-01,-150.0,USD,-0.07,int_c,CBD US
6,038CDB895,ms,2018-05-01,-300.0,USD,-0.10,int_c,SID US
7,038CDB895,ms,2018-05-01,-210.0,USD,-0.94,int_c,PAC US
8,038CDB895,ms,2018-05-01,-1500.0,USD,-0.45,int_c,ITUB US
9,038CDB895,ms,2018-05-01,-13700.0,USD,-7.62,int_c,LTM US


In [47]:
pvt = check_days(calypso_month, pb_month)
pvt

check                                                
tipo       calypso                         pb                     
acc          int_c ms_cayman ubs_cayman int_c ms_cayman ubs_cayman
data_ref                                                          
2018-05-01       1         1          1     1         1          1
2018-05-02       1         1          1     1         1          1
2018-05-03       1         1          1     1         1          1
2018-05-04       1         1          1     1         1          1
2018-05-07       1         1          1     1         1          1
2018-05-08       1         1          1     1         1          1
2018-05-09       1         1          1     1         1          1
2018-05-10       1         1          1     1         1          1
2018-05-11       1         1          1     1         1          1
2018-05-14       1         1          1     1         1          1
2018-05-15       1         1          1     1         1          1
2018-05-16       1         1          1     1         1          1
2018-05-17       1         1          1     1         1          1
2018-05-18       1         1          1     1         1          1
2018-05-21       1         1          1     1         1          1
2018-05-22       1         1          1     1         1          1
2018-05-23       1         1          1     1         1          1
2018-05-24       1         1          1     1         1          1
2018-05-25       1         1          1     1         1          1
2018-05-28       1         1          1     1         1          1
2018-05-29       1         1          1     1         1          1
2018-05-30       1         1          1     1         1          1
2018-05-31       1         1          1     1         1          1

In [48]:
pb_month, calypso_batimento = ajustes_pb_calypso_month(pb_month, calypso_month, dict_contas)
try:
    pb_month_acumulado, calypso_batimento_acumulado = \
                            ajustes_pb_calypso_month(pb_month_acumulado, calypso_month_acumulado, dict_contas)
except:
    print ('Arquivos não acumuláveis')

Arquivos não acumuláveis


In [98]:
pb_month.groupby(by=['acc','currency']).sum()

qt  accrual_adjusted
acc        currency                               
int_c      AUD       -13057100.0          -2797.75
           MXN         -234600.0          -1663.57
           USD       -36590951.0         -20437.24
ms_cayman  EUR         -748223.0           -467.13
           GBP            -690.0             -1.16
           MXN      -470027247.0        -982264.52
           USD      -151990449.0        -148173.57
ubs_cayman MXN      -241427869.0       -1016748.48
           USD       -70295270.0          33339.25

In [99]:
pb_month.shape

(4367, 6)

In [49]:
sobra_destino, sobra_origem, batido, divergentes, corrigidos, n_batidos, distribuidos = \
                                            batimento_calypso_pb(pb_month, calypso_batimento, ticker_diffs)

7.007098694756131 pct dos trades estão no prime broker, mas não acharam correspondência no Calypso
0.9028794533918985 pct dos trades foram achados no calypso, mas não têm correspondência no prime broker


In [50]:
try:
    sobra_destino_acumulado, sobra_origem_acumulado, batido_acumulado, divergentes_acumulado, corrigidos_acumulado,\
    n_batidos_acumulado, distribuidos_acumulado = \
                batimento_calypso_pb(pb_month_acumulado, calypso_batimento_acumulado, ticker_diffs)
except:
    print ('Arquivos não acumuláveis')

Arquivos não acumuláveis


In [97]:
distribuidos.groupby(by=['acc','currency']).sum()

qt  accrual_adjusted   quantidade         id
acc        currency                                                       
int_c      AUD       -13057100.0          -2797.75  -13057100.0     3481.0
           MXN         -234600.0          -1663.57    -234600.0     3504.0
           USD       -36590951.0         -20437.24  -36590951.0    43988.0
ms_cayman  EUR         -748223.0           -467.13    -748223.0    79302.0
           GBP            -690.0             -1.16       -690.0    38694.0
           MXN      -470027247.0        -982264.52 -470027247.0   191411.0
           USD      -151888532.0        -148157.11 -144542802.0  4157615.0
ubs_cayman MXN      -233690065.0        -979912.33 -232459688.0  1110141.0
           USD       -70032385.0          33401.42  -70009921.0  2688046.0

### Distribuição por books

In [51]:
#Pega as informações do calypso para o mês de referência. 
calypso_month_books, calypso_month_books_acumulado = get_calypso_month_books(calypso,mes_ref, ano_ref, data, referencia = tipo)

Você está fazendo a distribuição referente a mai/2018


In [52]:
calypso_month_books, abertura, abertura_n_batidos = explode_books(calypso_month_books, calypso_batimento, distribuidos)

Ajustes na base calypso estão completos
Ajustes de quantidades estão completos
Checando quais trades ficaram fora do batimento


In [53]:
try:
    calypso_month_books_acumulado, abertura_acumulado, abertura_n_batidos_acumulado = \
                        explode_books(calypso_month_books_acumulado, calypso_batimento_acumulado, distribuidos_acumulado)
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')

calypso_month_books_acumulado provavelmente está vazio


In [54]:
abertura

,conta_pb,conta,agent,data_ref,quantidade_book,book,ticker,isin,cusip,currency,month,year,acc,quantidade_ticker,qt,accrual_adjusted,quantidade,id,status,quantidade_conta_pb
0,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-5331.0,13F_CONS_N_CYCL.CAYMAN,RDUS US,US7504692077,750469207,USD,mai,2018,ms_cayman,-5331.0,-5331.0,-11.07,-5331.0,402.0,batido,-5331.0
1,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-9434.0,13F_INDU.CAYMAN,SBLK US,MHY8162K2046,Y8162K105,USD,mai,2018,ms_cayman,-9434.0,-9434.0,-9.05,-9434.0,409.0,batido,-9434.0
2,038CDCN39,ASSET,MORGAN STANLEY & CO,2018-05-01,-203208.0,MEX_DIRECIONAL.CAYMAN,BSMX US,US05969B1035,05969B103,USD,mai,2018,ms_cayman,-203208.0,-203208.0,-31.03,-203208.0,339.0,batido,-203208.0
3,038CDCN39,ASSET,MORGAN STANLEY & CO,2018-05-01,-2888800.0,MEX_DIRECIONAL.CAYMAN,AMX US,US02364W1053,02364W105,USD,mai,2018,ms_cayman,-3037969.0,-3037969.0,-1153.79,-3037969.0,322.0,batido,-2888800.0
4,75202213,ASSET,UBS SECURITIES LLC,2018-05-01,-9930.0,SICAV LATAM HEDGE.CAYMAN_UBS,ECH US,US4642866408,464286640,USD,mai,2018,ubs_cayman,-40057.0,-40057.0,-88.70,-40057.0,3034.0,batido,-9930.0
5,75202213,ASSET,UBS SECURITIES LLC,2018-05-01,-70510.0,SICAV LATAM HEDGE.CAYMAN_UBS,EWZ US,US4642864007,464286400,USD,mai,2018,ubs_cayman,-70510.0,-70510.0,102.26,-70510.0,3036.0,batido,-70510.0
6,038CDCN39,ASSET,MORGAN STANLEY & CO,2018-05-01,-30036.0,MEX_DIRECIONAL.CAYMAN,ASR US,US40051E2028,40051E202,USD,mai,2018,ms_cayman,-30036.0,-30036.0,-190.77,-30036.0,329.0,batido,-30036.0
7,75202213,ASSET,UBS SECURITIES LLC,2018-05-01,-36660.0,SICAV LATAM HEDGE.CAYMAN_UBS,EWW US,US4642868222,464286822,USD,mai,2018,ubs_cayman,-1001297.0,-1001297.0,-354.00,-1001297.0,3035.0,batido,-36660.0
8,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-4335432.0,LATAM_TELECOM.CAYMAN,AMXL MM,MXP001691213,NaN,MXN,mai,2018,ms_cayman,-4335432.0,-4335432.0,-2244.13,-4335432.0,323.0,batido,-4335432.0
10,038CDBD08,ASSET,MORGAN STANLEY & CO,2018-05-01,-201211.0,13F_INDU.CAYMAN,DHT US,MHY2065G1219,NaN,USD,mai,2018,ms_cayman,-201211.0,-201211.0,-15.30,-201211.0,351.0,batido,-201211.0


In [55]:
abertura_n_batidos

,conta_pb,conta,agent,data_ref,quantidade_book,book,ticker,isin,cusip,currency,month,year,acc,quantidade_ticker,qt,accrual_adjusted,quantidade,id,status,quantidade_conta_pb


In [56]:
abertura, abertura_batidos, abertura_n_batidos, bate_soma, bate_soma_pb, check = \
                                        ajuste_abertura_checs(abertura, abertura_n_batidos, pb_month)

Ajustes nas quantidades e alocação percentual feitos
Ajustes nas bases feitos
Geradas os dataframes de batimento
Dataframe de batimento gerado


In [57]:
try:
    abertura_acumulado, abertura_batidos_acumulado, abertura_n_batidos_acumulado, bate_soma_acumulado, bate_soma_pb_acumulado, \
                                    check_acumulado =  \
                            ajuste_abertura_checs(abertura_acumulado, abertura_n_batidos_acumulado, pb_month_acumulado)
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')

calypso_month_books_acumulado provavelmente está vazio


In [96]:
check.drop(['quantidade_pb','quantidade_calypso','bate_qt'],axis=1)

,acc,currency,custos_pb,custos_batidos,bate_custos,bate_custos_pct
0,int_c,AUD,-2797.75,-2797.75,0.000000e+00,1.000000
1,int_c,MXN,-1663.57,-1663.57,0.000000e+00,1.000000
2,int_c,USD,-20437.24,-20437.24,-7.275958e-12,1.000000
3,ms_cayman,EUR,-467.13,-467.13,1.136868e-13,1.000000
4,ms_cayman,GBP,-1.16,-1.16,0.000000e+00,1.000000
5,ms_cayman,MXN,-982264.52,-982264.52,-2.328306e-10,1.000000
6,ms_cayman,USD,-148173.57,-148157.11,-1.646000e+01,1.000111
7,ubs_cayman,MXN,-1016748.48,-979912.33,-3.683615e+04,1.037591
8,ubs_cayman,USD,33339.25,33401.42,-6.217000e+01,0.998139


### Abertura por WM

In [59]:
#from aluguel_offshore import *

In [67]:
pb_wm = abertura[(abertura['book'].str.contains('PRIVATE'))|(abertura['book'].str.contains('CLIENT'))]

In [68]:
try:
    pb_wm_acumulado = abertura_acumulado[(abertura_acumulado['book'].str.contains('PRIVATE'))\
                                         |(abertura_acumulado['book'].str.contains('CLIENT'))]
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')    

calypso_month_books_acumulado provavelmente está vazio


In [69]:
calypso_wm, calypso_wm_grouped, calypso_wm_acumulado, calypso_wm_grouped_acumulado = \
                                ajuste_wm(bdwm, data ,mes_ref, ano_ref, referencia = tipo)

Sem valores de acumulado


In [70]:
batimento_wm = pd.merge(pb_wm, calypso_wm_grouped, on=['data_ref', 'book', 'ticker', 'currency'], how='left')\
                                                                                    .drop('year', axis=1)

try:
    batimento_wm_acumulado = pd.merge(pb_wm_acumulado, calypso_wm_grouped_acumulado, \
                                      on=['data_ref', 'book', 'ticker', 'currency'], how='left').drop('year', axis=1)
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')

calypso_month_books_acumulado provavelmente está vazio


In [71]:
batimento_wm_distrib, abertura_batidos_final, n_batimento_wm_distrib, n_distrib_wm, batimento_wm_sobra_origem = \
                                    get_batimento_wm_distb(batimento_wm, calypso_wm, abertura_batidos)
    
abertura_batidos_final = abertura_batidos_final.rename(columns={
    'conta':'arquivo_origem',
    'conta_pb':'conta'    
}).reset_index(drop=True)

C:\Users\correigu\Python\aluguel_offshore\aluguel_offshore.py:325: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  batimento_wm_sobra_origem = batimento_wm_batidos[~batimento_wm_distrib['id'].isin(batimento_wm_batidos['id'].drop_duplicates())]


NameError: name 'abertura_batidos_final' is not defined

In [72]:
batimento_wm_distrib

NameError: name 'batimento_wm_distrib' is not defined

In [ ]:
try:
    batimento_wm_distrib_acumulado, abertura_batidos_final_acumulado, n_batimento_wm_distrib_acumulado, n_distrib_wm_acumulado = \
                                    get_batimento_wm_distb(batimento_wm_acumulado, calypso_wm_acumulado, \
                                                           abertura_batidos_acumulado)
        
    abertura_batidos_final_acumulado = abertura_batidos_final_acumulado.rename(columns={
    'conta':'arquivo_origem',
    'conta_pb':'conta'    
}).reset_index(drop=True)
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')

In [73]:
n_batimento_wm_distrib

NameError: name 'n_batimento_wm_distrib' is not defined

In [74]:
abertura_batidos_consol = abertura_batidos.groupby(['acc', 'book', 'currency'])\
                                        .sum().reset_index().drop('quantidade_book', axis=1)


In [75]:
total_wm = pd.DataFrame(batimento_wm_distrib.groupby(by=['acc','book','currency']).sum()['accrual_final']).reset_index()

NameError: name 'batimento_wm_distrib' is not defined

In [76]:
total_wm_calypso = abertura_batidos_consol[(abertura_batidos_consol['book'].str.contains('CLIENTS'))|\
                       (abertura_batidos_consol['book'].str.contains('PRIVATE'))].reset_index(drop=True)

In [77]:
total_wm

NameError: name 'total_wm' is not defined

In [78]:
total_wm_calypso

,acc,book,currency,accrual_final
0,ubs_cayman,BD_CLIENTS_UBS,USD,1343.423491
1,ubs_cayman,PRIVATE_DEFAULT_UBS,USD,-9138.164422


In [79]:
total_wm_n_distrib = pd.merge(total_wm, total_wm_calypso, on=['acc','book','currency'], how='left')

NameError: name 'total_wm' is not defined

In [80]:
total_wm_n_distrib['accrual_final'] = total_wm_n_distrib['accrual_final_x']-total_wm_n_distrib['accrual_final_y']

NameError: name 'total_wm_n_distrib' is not defined

In [81]:
total_wm_n_distrib = total_wm_n_distrib.drop(['accrual_final_x', 'accrual_final_y'], axis=1)

NameError: name 'total_wm_n_distrib' is not defined

In [82]:
total_wm_n_distrib

NameError: name 'total_wm_n_distrib' is not defined

In [83]:
abertura_batidos_consol = pd.concat([abertura_batidos_consol, total_wm_n_distrib], axis=0)\
                            .groupby(by=['acc','book','currency']).sum().reset_index()

NameError: name 'total_wm_n_distrib' is not defined

In [84]:
abertura_batidos_consol

,acc,book,currency,accrual_final
0,int_c,ACOES_OFFSHORE_.INTC,AUD,-2797.750000
1,int_c,ACOES_OFFSHORE_.INTC,MXN,-1663.570000
2,int_c,ACOES_OFFSHORE_.INTC,USD,1524.049791
3,int_c,ACOES_OFFSHORE_INTC,USD,-1959.812938
4,int_c,INTRADAY_PAIRS.INTC,USD,-163.723634
5,int_c,LOCAL_ADR_INTC,USD,-7638.912803
6,int_c,LOCAL_ADR_MR.INTC,USD,-12198.840416
7,ms_cayman,13F_COMM.CAYMAN,USD,-3452.080000
8,ms_cayman,13F_CONS_CYCL.CAYMAN,USD,-878.135495
9,ms_cayman,13F_CONS_N_CYCL.CAYMAN,USD,-1159.173843


In [85]:
batimento_wm_distrib = batimento_wm_distrib.drop('accrual_n_distrib', axis=1)

NameError: name 'batimento_wm_distrib' is not defined

In [86]:
abertura_batidos_consol_ajuste = abertura_batidos_consol.groupby(by=['acc', 'currency']).sum().reset_index()

In [87]:
'''info_emails = pd.DataFrame({
    'acc':['ms_cayman','ms_cayman','ms_cayman','ubs_cayman', 'int_c','int_c','int_c'],
    'currency':['USD','GBP','MXN','MXN','AUD','USD','MXN'],
    'accrual_final':[-152710.77,-1.07,-999326.27,-721247.90,-2492.32,-9907.02,-1482.73]
})'''

"info_emails = pd.DataFrame({\n    'acc':['ms_cayman','ms_cayman','ms_cayman','ubs_cayman', 'int_c','int_c','int_c'],\n    'currency':['USD','GBP','MXN','MXN','AUD','USD','MXN'],\n    'accrual_final':[-152710.77,-1.07,-999326.27,-721247.90,-2492.32,-9907.02,-1482.73]\n})"

In [88]:
'''info_emails = pd.DataFrame({
    'acc':['ms_cayman','ms_cayman','ms_cayman','ubs_cayman','ubs_cayman', 'int_c','int_c'],
    'currency':['USD','GBP','MXN','USD','MXN','AUD','USD'],
    'accrual_final':[-193547.92, -1.23, -410018.67,-32656.22,-747445.48,-2290.07,-2600.76]
})'''

"info_emails = pd.DataFrame({\n    'acc':['ms_cayman','ms_cayman','ms_cayman','ubs_cayman','ubs_cayman', 'int_c','int_c'],\n    'currency':['USD','GBP','MXN','USD','MXN','AUD','USD'],\n    'accrual_final':[-193547.92, -1.23, -410018.67,-32656.22,-747445.48,-2290.07,-2600.76]\n})"

In [89]:
info_emails = pd.DataFrame({
    'acc':['ms_cayman','ms_cayman','ms_cayman','ms_cayman','ubs_cayman', 'int_c','int_c','int_c'],
    'currency':['USD','EUR','GBP','MXN','MXN','AUD','USD','MXN'],
    'accrual_final':[-149037.61,-467.13,-1.16,-982264.52,-1037265.09,-2797.75,-20443.32,-1663.57]
})

In [90]:
info_emails

,acc,accrual_final,currency
0,ms_cayman,-149037.61,USD
1,ms_cayman,-467.13,EUR
2,ms_cayman,-1.16,GBP
3,ms_cayman,-982264.52,MXN
4,ubs_cayman,-1037265.09,MXN
5,int_c,-2797.75,AUD
6,int_c,-20443.32,USD
7,int_c,-1663.57,MXN


In [91]:
ajuste = pd.merge(abertura_batidos_consol_ajuste, info_emails, on=['acc','currency'], how='left')

In [92]:
ajuste['multiplicador'] = ajuste['accrual_final_y']/ajuste['accrual_final_x']

In [93]:
ajuste = ajuste.rename(columns={
    'accrual_final_x':'accrual_final_gerado',
    'accrual_final_y':'accrual_final_email' 
}).fillna(0)

In [94]:
ajuste

,acc,currency,accrual_final_gerado,accrual_final_email,multiplicador
0,int_c,AUD,-2797.75,-2797.75,1.000000
1,int_c,MXN,-1663.57,-1663.57,1.000000
2,int_c,USD,-20437.24,-20443.32,1.000297
3,ms_cayman,EUR,-467.13,-467.13,1.000000
4,ms_cayman,GBP,-1.16,-1.16,1.000000
5,ms_cayman,MXN,-982264.52,-982264.52,1.000000
6,ms_cayman,USD,-147429.73,-149037.61,1.010906
7,ubs_cayman,MXN,-891483.31,-1037265.09,1.163527
8,ubs_cayman,USD,33378.34,0.00,0.000000


In [404]:
ajuste = ajuste.drop(['accrual_final_gerado', 'accrual_final_email'], axis=1)

In [405]:
abertura_batidos_consol = pd.merge(abertura_batidos_consol, ajuste, on=['acc','currency'], how='left')

In [406]:
abertura_batidos_consol['accrual'] = abertura_batidos_consol['accrual_final']*abertura_batidos_consol['multiplicador']

In [407]:
abertura_batidos_consol = abertura_batidos_consol.drop(['accrual_final','multiplicador'], axis=1)

In [408]:
abertura_batidos_consol = abertura_batidos_consol[abertura_batidos_consol['accrual']!=0]

In [409]:
abertura_batidos_consol

,acc,book,currency,accrual
0,int_c,ACOES_OFFSHORE_.INTC,AUD,-2.797750e+03
1,int_c,ACOES_OFFSHORE_.INTC,MXN,-1.663570e+03
2,int_c,ACOES_OFFSHORE_.INTC,USD,1.524503e+03
3,int_c,ACOES_OFFSHORE_INTC,USD,-1.960396e+03
4,int_c,INTRADAY_PAIRS.INTC,USD,-1.637723e+02
5,int_c,LOCAL_ADR_INTC,USD,-7.641185e+03
6,int_c,LOCAL_ADR_MR.INTC,USD,-1.220247e+04
7,ms_cayman,13F_COMM.CAYMAN,USD,-3.489729e+03
8,ms_cayman,13F_CONS_CYCL.CAYMAN,USD,-8.877125e+02
9,ms_cayman,13F_CONS_N_CYCL.CAYMAN,USD,-1.171816e+03


In [410]:
batimento_wm_distrib = pd.merge(batimento_wm_distrib, ajuste, on=['acc','currency'], how='left')

In [411]:
batimento_wm_distrib['accrual'] = batimento_wm_distrib['accrual_final']*batimento_wm_distrib['multiplicador']

In [412]:
batimento_wm_distrib = batimento_wm_distrib.drop(['accrual_final', 'multiplicador'], axis=1)

In [413]:
batimento_wm_distrib = batimento_wm_distrib[batimento_wm_distrib['accrual']!=0]

# Resultados

In [414]:
string = get_name(mes_ref, ano_ref, data, referencia = tipo)

In [415]:
save_path = r'\\DRIOC0231PFS\Apoio_SB\Geral\BD\Aluguel_Offshore\Arquivos diários'

In [416]:
if tipo == 'month':
    subfolder = mes_ref+str(ano_ref)
    final_path = save_path+'\\'+subfolder+'\\'+'consolidado'
    if os.path.isdir(save_path+'\\'+subfolder)==True:
        if os.path.isdir(save_path+'\\'+subfolder+'\\'+'consolidado')==False:
            os.mkdir(save_path+'\\'+subfolder+'\\'+'consolidado')
    else:
        os.mkdir(save_path+'\\'+subfolder)
        os.mkdir(save_path+'\\'+subfolder+'\\'+'consolidado')
else:
    subfolder = months_dict[pd.to_datetime(data).month]+str(pd.to_datetime(data).year)
    final_path = save_path+'\\'+subfolder+'\\'+data
    if os.path.isdir(save_path+'\\'+subfolder)==True:
        if os.path.isdir(save_path+'\\'+subfolder+'\\'+data)==False:
            os.mkdir(save_path+'\\'+subfolder+'\\'+data)
    else:
        os.mkdir(save_path+'\\'+subfolder)
        os.mkdir(save_path+'\\'+subfolder+'\\'+data)

In [417]:
final_path

'\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\mai2018\\consolidado'

In [418]:
writer = pd.ExcelWriter(final_path+'\\batimento_alguel_offshore_'+string+'.xlsx')

In [419]:
'''abertura_batidos_final.to_excel(writer, sheet_name = 'abertura_batidos_final')

batimento_wm_distrib.to_excel(writer, sheet_name = 'batimento_wm_distrib')

abertura_n_batidos.to_excel(writer, sheet_name = 'abertura_n_batidos')

sobra_origem.to_excel(writer, sheet_name = 'sobra_origem')

sobra_destino.to_excel(writer, sheet_name = 'sobra_destino')'''

"abertura_batidos_final.to_excel(writer, sheet_name = 'abertura_batidos_final')\n\nbatimento_wm_distrib.to_excel(writer, sheet_name = 'batimento_wm_distrib')\n\nabertura_n_batidos.to_excel(writer, sheet_name = 'abertura_n_batidos')\n\nsobra_origem.to_excel(writer, sheet_name = 'sobra_origem')\n\nsobra_destino.to_excel(writer, sheet_name = 'sobra_destino')"

In [420]:
abertura_batidos_consol.to_excel(writer, sheet_name = 'Custos Consolidados')

In [421]:
batimento_wm_distrib.to_excel(writer, sheet_name = 'Distrib WM')

In [422]:
writer.save()

In [423]:
writer_2 = pd.ExcelWriter(final_path+'\\batimento_alguel_offshore_sobras'+string+'.xlsx')

In [424]:
abertura_n_batidos.to_excel(writer_2, sheet_name = 'abertura_n_batidos')
sobra_origem.to_excel(writer_2, sheet_name = 'sobra_origem')
sobra_destino.to_excel(writer_2, sheet_name = 'sobra_destino')

In [425]:
writer_2.save()

In [ ]:
abertura_batidos_final
abertura_batidos_final_email = abertura_batidos_final.groupby(['acc', 'book', 'currency', 'arquivo_origem'])\
                                        .sum().reset_index().drop('quantidade_book', axis=1)

try:
    abertura_batidos_final_acumulado
    abertura_batidos_final_acumulado_email = abertura_batidos_final_acumulado\
                                        .groupby(['acc', 'book', 'currency', 'arquivo_origem'])\
                                            .sum().reset_index().drop('quantidade_book', axis=1)
except:
    print ('calypso_month_books_acumulado provavelmente está vazio')

In [ ]:
abertura_batidos_final_email.to_excel(writer, sheet_name = 'abertura_batidos_final_email')

In [ ]:
n_batimento_wm_distrib[n_batimento_wm_distrib['currency']!='CLP']

In [ ]:
pb_wm.sum()

In [ ]:
if abertura_batidos_final_acumulado_email.dropna().shape[0]>2:
    writer_1 = pd.ExcelWriter(final_path+'\\batimento_alguel_offshore_acumulado'+string+'.xlsx')
    abertura_batidos_final_acumulado_email.to_excel(writer_1, sheet_name = 'consolidado')
    writer.save()